In [3]:
import redis
import pickle
import time
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq
from io import StringIO
from datetime import datetime
from typing import Dict, Optional

class MinuteDataProcessor:
    def __init__(self, config_path: str = "redis.conf"):
        """初始化处理器，与发布器保持相同的Redis配置逻辑"""
        self.redis_config = self._load_redis_config(config_path)
        self.redis = redis.Redis(
            host=self.redis_config["host"],
            port=self.redis_config["port"],
            password=self.redis_config["password"],
            decode_responses=False
        )
        self.result_queue = "function_results"  # 与发布器对应
        self.task_metadata = "task_metadata"     # 与发布器存储元信息的键一致
        self.storage_root = r"D:\workspace\xiaoyao\data\stock_minutely_price"
        self.idle_timeout = 1800  # 30分钟无任务退出
        self._test_redis_connection()
        self._init_storage()

    def _load_redis_config(self, config_path: str) -> Dict[str, str]:
        """复用发布器的Redis配置加载逻辑，确保一致"""
        config = {"host": "localhost", "port": 6379, "password": ""}
        try:
            with open(config_path, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line.startswith("host="):
                        config["host"] = line.split("=", 1)[1].strip()
                    elif line.startswith("port="):
                        config["port"] = int(line.split("=", 1)[1].strip())
                    elif line.startswith("password="):
                        config["password"] = line.split("=", 1)[1].strip()
            return config
        except Exception as e:
            print(f"⚠️ 配置文件读取失败，使用默认配置: {e}")
            return config

    def _test_redis_connection(self):
        """测试Redis连接，与发布器逻辑一致"""
        try:
            self.redis.ping()
            print(f"✅ 处理器Redis连接成功 | {self.redis_config['host']}:{self.redis_config['port']}")
        except Exception as e:
            print(f"❌ 处理器Redis连接失败: {e}")
            raise SystemExit(1)

    def _init_storage(self):
        """初始化存储目录"""
        os.makedirs(self.storage_root, exist_ok=True)
        print(f"✅ 数据存储目录: {self.storage_root}")

    def _process_csv_data(self, csv_str: str, task_id: str) -> bool:
        """处理CSV数据并按股票存储（仅按股票分区）"""
        if not csv_str.strip():
            print(f"⚠️ 任务{task_id}返回空数据，跳过处理")
            return False

        try:
            # 从CSV字符串读取数据（与发布器的CSV处理逻辑兼容）
            df = pd.read_csv(StringIO(csv_str))
            
            # 必要字段校验
            required_cols = ['date', 'stock_code', 'time', 'open', 'close', 'high', 'low', 'volume']
            missing_cols = [col for col in required_cols if col not in df.columns]
            if missing_cols:
                raise ValueError(f"缺少必要字段: {missing_cols}")

            # 按股票代码存储（仅股票分区，无日期分区）
            for stock_code in df['stock_code'].unique():
                stock_data = df[df['stock_code'] == stock_code].copy()
                stock_dir = os.path.join(self.storage_root, f"stock_code={stock_code}")
                os.makedirs(stock_dir, exist_ok=True)
                parquet_path = os.path.join(stock_dir, "data.parquet")

                # 转换为Arrow表并追加/创建文件
                table = pa.Table.from_pandas(stock_data)
                if os.path.exists(parquet_path):
                    existing_table = pq.read_table(parquet_path)
                    combined_table = pa.concat_tables([existing_table, table])
                    pq.write_table(combined_table, parquet_path, compression="snappy")
                else:
                    pq.write_table(table, parquet_path, compression="snappy")

            return True
        except Exception as e:
            print(f"❌ 任务{task_id}数据处理失败: {str(e)[:100]}")
            return False

    def listen_and_process(self):
        """监听结果队列并处理数据"""
        print(f"✅ 开始监听结果队列（{self.idle_timeout}秒无任务退出）")
        stats = {"success": 0, "failed": 0, "last_active": time.time()}

        while True:
            # 检查超时退出
            if time.time() - stats["last_active"] > self.idle_timeout:
                print("\n⏰ 长时间无新任务，退出处理器")
                break

            # 从队列获取结果（与发布器的序列化方式匹配）
            try:
                _, result_bytes = self.redis.blpop(self.result_queue, timeout=30)
                if not result_bytes:
                    continue  # 无数据，继续等待

                # 更新活动时间
                stats["last_active"] = time.time()

                # 反序列化结果（使用pickle，与发布器一致）
                result = pickle.loads(result_bytes)
                task_id = result.get("task_id", "未知任务")

                # 处理结果（与发布器的任务结构对应）
                if result.get("status") == "success":
                    # 处理成功结果
                    csv_data = result.get("result", "")
                    if self._process_csv_data(csv_data, task_id):
                        stats["success"] += 1
                        # 清理任务元信息（与发布器存储的元信息键对应）
                        self.redis.hdel(self.task_metadata, task_id)
                        print(f"✅ 任务{task_id[:8]}...处理成功 | 累计成功: {stats['success']}")
                    else:
                        stats["failed"] += 1
                        print(f"❌ 任务{task_id[:8]}...数据处理失败 | 累计失败: {stats['failed']}")
                else:
                    # 处理远程执行失败的任务
                    stats["failed"] += 1
                    error_msg = result.get("error", "无错误信息")
                    print(f"❌ 任务{task_id[:8]}...远程执行失败: {error_msg} | 累计失败: {stats['failed']}")

            except Exception as e:
                print(f"⚠️ 处理器异常: {str(e)[:80]}，等待10秒重试")
                time.sleep(10)

        # 输出最终统计
        print("\n" + "="*50)
        print("结果处理总结")
        print(f"总处理任务数: {stats['success'] + stats['failed']}")
        print(f"成功: {stats['success']} | 失败: {stats['failed']}")
        if stats["success"] + stats["failed"] > 0:
            print(f"成功率: {stats['success']/(stats['success']+stats['failed'])*100:.1f}%")
        print("="*50)

if __name__ == "__main__":
    try:
        processor = MinuteDataProcessor(config_path="redis.conf")
        processor.listen_and_process()
    except Exception as e:
        print(f"❌ 处理器执行失败: {e}")


✅ 处理器Redis连接成功 | 220.203.1.124:6379
✅ 数据存储目录: D:\workspace\xiaoyao\data\stock_minutely_price
✅ 开始监听结果队列（1800秒无任务退出）
✅ 任务task_6f8...处理成功 | 累计成功: 1
✅ 任务task_be4...处理成功 | 累计成功: 2
✅ 任务task_b90...处理成功 | 累计成功: 3
✅ 任务task_3e5...处理成功 | 累计成功: 4
✅ 任务task_9fb...处理成功 | 累计成功: 5
✅ 任务task_ee9...处理成功 | 累计成功: 6
✅ 任务task_f5c...处理成功 | 累计成功: 7
✅ 任务task_45b...处理成功 | 累计成功: 8
✅ 任务task_db0...处理成功 | 累计成功: 9
✅ 任务task_d93...处理成功 | 累计成功: 10
✅ 任务task_ada...处理成功 | 累计成功: 11
✅ 任务task_2ac...处理成功 | 累计成功: 12
✅ 任务task_310...处理成功 | 累计成功: 13
✅ 任务task_6d0...处理成功 | 累计成功: 14
✅ 任务task_788...处理成功 | 累计成功: 15
✅ 任务task_a90...处理成功 | 累计成功: 16
✅ 任务task_36b...处理成功 | 累计成功: 17
✅ 任务task_7df...处理成功 | 累计成功: 18
✅ 任务task_621...处理成功 | 累计成功: 19
✅ 任务task_403...处理成功 | 累计成功: 20
✅ 任务task_afd...处理成功 | 累计成功: 21
✅ 任务task_c9e...处理成功 | 累计成功: 22
✅ 任务task_7d2...处理成功 | 累计成功: 23
✅ 任务task_c02...处理成功 | 累计成功: 24
✅ 任务task_933...处理成功 | 累计成功: 25
✅ 任务task_5f9...处理成功 | 累计成功: 26
✅ 任务task_40b...处理成功 | 累计成功: 27
✅ 任务task_408...处理成功 | 累计成功: 28
✅ 任务task_87f...处理成功 | 累计成

KeyboardInterrupt: 